# New House location analysis according to closeness to metro stations


In [114]:
import numpy as np
import pandas as pd

import requests
import urllib.request
import csv
import json

from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

In [225]:
from geopy.geocoders import Nominatim
import folium 

#address = 'Chicago, USA'
address = 'Moscow, RUS'

geolocator = Nominatim(user_agent="my-app")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of city are 55.7504461, 37.6174943.


In [226]:
# @hidden_cell
CLIENT_ID = 'ACZJOECQKRZ2W0XWIZRIOUW4YGI3030QJWHVTMVZZKLH0AMN' # your Foursquare ID
CLIENT_SECRET = 'E50W0Y52T0RTMTAQDUSMW02AFR5YANAIQIUKEY2PAGY4XP4K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [227]:
radius = 15000
n_results = 500
# Theater
#cat_id = '4bf58dd8d48988d137941735'
# Metro station
cat_id = '4bf58dd8d48988d1fd931735'

# beer house
url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#url = 'https://api.foursquare.com/v2/venues/search?&categoryId=4bf58dd8d48988d137941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    cat_id,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    n_results)
print(url)

https://api.foursquare.com/v2/venues/search?&categoryId=4bf58dd8d48988d1fd931735&client_id=ACZJOECQKRZ2W0XWIZRIOUW4YGI3030QJWHVTMVZZKLH0AMN&client_secret=E50W0Y52T0RTMTAQDUSMW02AFR5YANAIQIUKEY2PAGY4XP4K&v=20180605&ll=55.7504461,37.6174943&radius=15000&limit=500


In [228]:
results = requests.get(url).json()

In [229]:
results['response']['venues']
venues = json_normalize(results['response']['venues'])
type(venues)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


pandas.core.frame.DataFrame

In [230]:
print(venues.columns)
filtered_columns = ['name', 'location.lat', 'location.lng']
venues = venues.loc[:, filtered_columns]

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.crossStreet', 'location.lat', 'location.lng',
       'location.labeledLatLngs', 'location.distance', 'location.postalCode',
       'location.cc', 'location.neighborhood', 'location.city',
       'location.state', 'location.country', 'location.formattedAddress'],
      dtype='object')


In [231]:
# clean columns
venues.columns = [col.split(".")[-1] for col in venues.columns]

venues.shape

(50, 3)

In [232]:
venues.tail(100)

,name,lat,lng
0,metro Kitay-gorod (Метро Китай-город),55.755196,37.633110
1,metro Tsvetnoy Bulvar (Метро Цветной бульвар),55.771525,37.620784
2,"metro Taganskaya, line 5 (Метро Таганская, кол...",55.742171,37.653771
3,metro Sokolniki (Метро Сокольники),55.789241,37.680353
4,metro Ploshchad Ilyicha (Метро Площадь Ильича),55.747179,37.681648
5,"metro Kiyevskaya, line 5 (Метро Киевская, коль...",55.743576,37.566194
6,"metro Belorusskaya, line 5 (Метро Белорусская,...",55.775309,37.582662
7,metro Proletarskaya (Метро Пролетарская),55.731690,37.664911
8,"metro Komsomolskaya, line 5 (Метро Комсомольск...",55.775596,37.654834
9,metro Krasnye Vorota (Метро Красные ворота),55.768659,37.649220


In [233]:
# create map of New York using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(venues['lat'], venues['lng'], venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

### Find and preprocess a list of houses under construction

In [351]:
flats = pd.read_json('data-24713-2020-06-04.json', encoding = "Windows-1251") #"ISO-8859-1")
flats.shape

(1579, 25)

In [352]:
flats.head()
filtered_columns = ['global_id', 'Coordinates', 'MainFunctional']
flats = flats.loc[:, filtered_columns]

In [353]:
df_clean = flats
#[flats['MainFunctional']=='Дом']
# Reindex
#df_clean.reset_index(drop=True, inplace=True)

In [354]:
df_clean

,global_id,Coordinates,MainFunctional
0,272715485,"37.693324502046,55.797677127859",Подземная стоянка
1,272715489,"37.489798795354,55.626083209178",Храм
2,272715492,"37.569607151073,55.685225196907",Офисное здание
3,272715504,"37.741970758303,55.690432838423","Объект аварийно-спасательной службы, формирования"
4,272715510,"37.595682531523,55.771000504538",Гостиница
...,...,...,...
1574,1037819557,"37.884342235382,55.724229565696",Объект торговли иного типа
1575,1037819558,"37.699581107495,55.697021718182",Больница
1576,1037862689,"37.474834237220,55.859468813591",Многоэтажный многоквартирный дом
1577,1037862798,"37.474834237220,55.859468813591",Многоэтажный многоквартирный дом


In [355]:
df_clean = df_clean[df_clean.apply(lambda row: row.astype(str).str.contains('Многоэтажный многоквартирный дом').any(), axis=1)]
df_clean.reset_index(drop=True, inplace=True)

In [356]:
df_clean

,global_id,Coordinates,MainFunctional
0,272726711,"37.795564262836,55.761349247059",Многоэтажный многоквартирный дом
1,272726777,"37.662497752668,55.606339142276",Многоэтажный многоквартирный дом
2,272726846,"37.677879903019,55.642152231422",Многоэтажный многоквартирный дом
3,272726887,"37.500611345616,55.801860524702",Многоэтажный многоквартирный дом
4,272726890,"37.561879230614,55.879979804897",Многоэтажный многоквартирный дом
...,...,...,...
453,1035998832,"37.609481885433,55.673467591528",Многоэтажный многоквартирный дом
454,1037160508,"37.539289973813,55.840327725186",Многоэтажный многоквартирный дом
455,1037862689,"37.474834237220,55.859468813591",Многоэтажный многоквартирный дом
456,1037862798,"37.474834237220,55.859468813591",Многоэтажный многоквартирный дом


In [357]:
#df_clean[['lat','lng']] = pd.DataFrame(df_clean['Coordinates'].astype(str).str.split(expand=True), index = df_clean.index)


df_clean[['lng','lat']] = df_clean['Coordinates'].astype(str).str.split(",", expand=True)
df_clean.drop(['Coordinates'], axis=1, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [358]:
df_clean

,global_id,MainFunctional,lng,lat
0,272726711,Многоэтажный многоквартирный дом,37.795564262836,55.761349247059
1,272726777,Многоэтажный многоквартирный дом,37.662497752668,55.606339142276
2,272726846,Многоэтажный многоквартирный дом,37.677879903019,55.642152231422
3,272726887,Многоэтажный многоквартирный дом,37.500611345616,55.801860524702
4,272726890,Многоэтажный многоквартирный дом,37.561879230614,55.879979804897
...,...,...,...,...
453,1035998832,Многоэтажный многоквартирный дом,37.609481885433,55.673467591528
454,1037160508,Многоэтажный многоквартирный дом,37.539289973813,55.840327725186
455,1037862689,Многоэтажный многоквартирный дом,37.474834237220,55.859468813591
456,1037862798,Многоэтажный многоквартирный дом,37.474834237220,55.859468813591


In [359]:
# create map of New York using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, gid in zip(df_clean['lat'], df_clean['lng'], df_clean['global_id']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
    
# add markers to map
for lat, lng, name in zip(venues['lat'], venues['lng'], venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)      
    
map

In [370]:
from sklearn.cluster import KMeans
kclusters = 50
km = KMeans(n_clusters=kclusters, init='random', max_iter=5, n_init=1)#, verbose=1)
km.fit(df_clean[['lat','lng']])

KMeans(init='random', max_iter=5, n_clusters=50, n_init=1)

In [371]:
km.labels_

array([41, 43, 42, 36, 45, 11, 43, 23,  5, 49, 17,  8, 43, 33, 34, 21, 43,
        3,  8, 49, 43, 45, 43, 41, 43, 43, 43, 43,  9, 47,  8, 18, 49,  1,
       16, 16, 16, 39, 39, 39, 16, 49,  7, 31, 44, 25, 17,  6, 25,  0, 16,
        7,  8, 25, 16, 49, 28,  8,  8, 21,  1, 25, 42, 19, 33,  7, 33, 17,
        8,  5, 33, 17, 33, 31, 18, 21, 37,  8, 17, 17, 17,  0,  3, 47, 21,
       10,  3,  3, 20, 29, 30, 37, 12,  8, 23, 40,  9,  6, 14, 40, 33, 14,
        9, 20, 21, 26, 20, 19, 22, 22, 10, 20, 22, 22, 20,  9, 30, 26, 30,
        0, 33, 16, 21, 11, 22, 21, 19,  0, 22, 29,  7,  0,  5,  9, 34, 23,
       40, 44, 10,  8, 30, 19, 29, 30,  9, 37, 20, 21, 12, 30, 34, 28, 10,
       40,  2, 20,  5, 40, 37,  4, 33,  0,  8, 12, 48, 10, 25, 25, 19, 31,
       38,  9, 48, 33, 47, 20,  9, 12,  7,  8, 20, 20,  6, 23, 34, 12, 16,
        9, 20, 11, 12,  8, 14, 22, 20, 10, 10,  9,  8, 47, 13, 20, 44, 20,
       22, 14, 10,  8,  7, 29, 31,  7, 28, 15, 22, 20, 48, 48, 10,  9, 31,
        6, 49, 49,  9, 14

In [372]:
df_clean['class'] = km.labels_

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [373]:
df_clean

,global_id,MainFunctional,lng,lat,class
0,272726711,Многоэтажный многоквартирный дом,37.795564262836,55.761349247059,41
1,272726777,Многоэтажный многоквартирный дом,37.662497752668,55.606339142276,43
2,272726846,Многоэтажный многоквартирный дом,37.677879903019,55.642152231422,42
3,272726887,Многоэтажный многоквартирный дом,37.500611345616,55.801860524702,36
4,272726890,Многоэтажный многоквартирный дом,37.561879230614,55.879979804897,45
...,...,...,...,...,...
453,1035998832,Многоэтажный многоквартирный дом,37.609481885433,55.673467591528,44
454,1037160508,Многоэтажный многоквартирный дом,37.539289973813,55.840327725186,30
455,1037862689,Многоэтажный многоквартирный дом,37.474834237220,55.859468813591,31
456,1037862798,Многоэтажный многоквартирный дом,37.474834237220,55.859468813591,31


In [374]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [375]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_clean['lat'], df_clean['lng'], df_clean['class']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters